In [1]:
import pickle
import torch

from src.dataSet import ProcessedDataSet
from src.load_csv import test_set
from src.preprocess import Preprocess
with open('../data/processed/preprocessed_trainSet.pkl', 'rb') as f:
    params = pickle.load(f)

C:\Users\sheng\.conda\envs\poisonous_mushrooms\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\sheng\.conda\envs\poisonous_mushrooms\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
#prepare validate data
preprocess = Preprocess(test_set)
validateSet = ProcessedDataSet(preprocess.processed, feature_order=params['feature_order'])
validateSet.enc_onehot = params['enc_onehot']
validateSet.enc_minmax = params['enc_minmax']
validateSet()
print(validateSet.sample.head(5))
print(validateSet.sample.shape)
print(validateSet[0])
print(validateSet.X.shape)

        id  cap-diameter cap-shape cap-surface cap-color does-bruise-or-bleed  \
0  3116945          8.64         x           t         n                    t   
1  3116946          6.90         o           t         o                    f   
2  3116947          2.00         b           g         n                    f   
3  3116948          3.47         x           t         n                    f   
4  3116949          6.17         x           h         y                    f   

  gill-attachment gill-spacing gill-color  stem-height  stem-width stem-color  \
0               a            c          w        11.13       17.12          w   
1               a            c          y         1.27       10.75          n   
2               a            c          n         6.18        3.14          n   
3               s            c          n         4.98        8.51          w   
4               p            c          y         6.73       13.70          y   

  has-ring ring-type habit

In [3]:
model_path = "../models/BinaryClassificationModel.pth"
output_file = "../data/processed/submission.csv"
batch_size = 32
input_dim = validateSet.sample.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
import pandas as pd
from src.predict import Prediction
from torch.utils.data import DataLoader

predictor = Prediction(validateSet.X,model_path)
test_loader = DataLoader(validateSet, batch_size=batch_size, shuffle=False)
model = predictor.load_model().to(device)
predictions = predictor.predict(model, test_loader, device)
ids = validateSet.sample['id'].values
submission = pd.DataFrame({
    'id': ids,
    'class': ['p' if p == 1 else 'e' for p in predictions]
})

In [5]:
submission.to_csv(output_file, index=False)
print(f"Predictions saved to {output_file}")

Predictions saved to ../data/processed/submission.csv


In [6]:
overview = pd.read_csv("../data/processed/submission.csv")
print(overview.head())

        id class
0  3116945     e
1  3116946     p
2  3116947     p
3  3116948     p
4  3116949     e
